In [1]:
# importing all needed libraries and api keys
import requests
import json
import re
import pandas as pd

from spotify_token_request import r

r

{'access_token': 'BQAbPxMNvd1oYGx_hz3ZuJWoSGHmjlpebK6ZgSWrq--L9qGHRqHkn4Bk4OgiN1ZTy-xUAFHPuf8ShobPBDMDh1UjRQGe9W9Pwr-lHHOrrUp8ahCCcL4',
 'expires_in': 3600,
 'token_type': 'Bearer'}


{'access_token': 'BQAbPxMNvd1oYGx_hz3ZuJWoSGHmjlpebK6ZgSWrq--L9qGHRqHkn4Bk4OgiN1ZTy-xUAFHPuf8ShobPBDMDh1UjRQGe9W9Pwr-lHHOrrUp8ahCCcL4',
 'token_type': 'Bearer',
 'expires_in': 3600}

In [2]:
# spotify information and headers for future api use
spotify_api_key = r['access_token']
spotify_headers = {
    'Authorization': 'Bearer {}'.format(spotify_api_key)
}

In [14]:
# reading csv and converting into a dictionary
csv_df = pd.read_csv('all_top100songs_by_year.csv')

all_songs_dictionary = {}

for x in range(0, 4500):
    all_songs_dictionary[x] = {}
    all_songs_dictionary[x]['title'] = csv_df['title'][x]
    all_songs_dictionary[x]['artist'] = csv_df['artist'][x]

all_songs_dictionary

{0: {'title': 'Lose Control', 'artist': 'Teddy Swims'},
 1: {'title': 'Stick Season', 'artist': 'Noah Kahan'},
 2: {'title': 'Beautiful Things', 'artist': 'Benson Boone'},
 3: {'title': 'Houdini', 'artist': 'Dua Lipa'},
 4: {'title': 'I Remember Everything',
  'artist': 'Zach Bryan Featuring Kacey Musgraves'},
 5: {'title': 'Saturn', 'artist': 'SZA'},
 6: {'title': 'Feather', 'artist': 'Sabrina Carpenter'},
 7: {'title': 'Austin', 'artist': 'Dasha'},
 8: {'title': 'Lovin On Me', 'artist': 'Jack Harlow'},
 9: {'title': 'Too Sweet', 'artist': 'Hozier'},
 10: {'title': 'Like That', 'artist': 'Future, Metro Boomin & Kendrick Lamar'},
 11: {'title': "Wind Up Missin' You", 'artist': 'Tucker Wetmore'},
 12: {'title': 'Slow It Down', 'artist': 'Benson Boone'},
 13: {'title': 'Good Luck, Babe!', 'artist': 'Chappell Roan'},
 14: {'title': 'Greedy', 'artist': 'Tate McRae'},
 15: {'title': 'A Bar Song (Tipsy)', 'artist': 'Shaboozey'},
 16: {'title': 'Espresso', 'artist': 'Sabrina Carpenter'},
 17:

In [15]:
# creating a new dictionary with edited artist values to not confuse the spotify search api, only takes the first artist when collaborations are formatted with "featured", "with", "x", etc...
search_dictionary = {}

for x in all_songs_dictionary:
    song = all_songs_dictionary[x]
    title = song['title']
    artist = song['artist']
    
    artist_edited = re.split(' Featuring | With | Duet | X | x | & |, | \(| And | Feat. ', artist)
    artist_edited = artist_edited[0]
    
    title_edited = re.split(' \(|/', title)
    title_edited = title_edited[0]
    title_edited = title_edited.replace("/"," ")
    
    search_dictionary[x] = {}
    search_dictionary[x]["title"] = title
    search_dictionary[x]["title_search"] = title_edited
    search_dictionary[x]["artist"] = artist
    search_dictionary[x]["artist_search"] = artist_edited

search_df = pd.DataFrame(search_dictionary)
search_df = search_df.transpose()
search_df.head(4500)

,title,title_search,artist,artist_search
0,Lose Control,Lose Control,Teddy Swims,Teddy Swims
1,Stick Season,Stick Season,Noah Kahan,Noah Kahan
2,Beautiful Things,Beautiful Things,Benson Boone,Benson Boone
3,Houdini,Houdini,Dua Lipa,Dua Lipa
4,I Remember Everything,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,Zach Bryan
...,...,...,...,...
4495,How Do I Survive,How Do I Survive,Amy Holland,Amy Holland
4496,Make A Little Magic,Make A Little Magic,The Dirt Band,The Dirt Band
4497,Old-Fashion Love,Old-Fashion Love,Commodores,Commodores
4498,The Wanderer,The Wanderer,Donna Summer,Donna Summer


In [16]:
# calling on spotify api to seach for each track's id, and adding the ids to a new dictionary
song_id_dictionary = {}
artist_split = []
 
for x in search_dictionary:
    song = search_dictionary[x]
    
    artist_split = song['artist_search'].split()
    artist = '+'.join(artist_split)
    
    title_split = song['title_search'].split()
    title = '+'.join(title_split)

    try:
        search_url = f"https://api.spotify.com/v1/search?q=track%3A{title}+artist%3A{artist}&type=track"

        search_response = requests.get(search_url, headers = spotify_headers).json()
        song_id_dictionary[x] = {}
        song_id_dictionary[x]['title'] = song['title']
        song_id_dictionary[x]['artist'] = song['artist']
        song_id_dictionary[x]['id'] = search_response['tracks']['items'][0]['id']
        
#        print(f"ID for song named {song['title']} added to list")

    except Exception as e: # this will return songs that the spotify api could not search correctly, and will have to be manually searched
        song_id_dictionary[x]['title'] = song['title']
        song_id_dictionary[x]['artist'] = song['artist']
        song_id_dictionary[x]['id'] = "n/a"
        print(f"no ID available for: {song['title']} by {song['artist']}")
        
    pass

no ID available for: Carnival by ¥$: Kanye West & Ty Dolla $ign
no ID available for: Hrs And Hrs by Muni Long
no ID available for: Ran$om by Lil Tecca
no ID available for: 1 Night by Lil Yachty
no ID available for: Rich As F**k by Lil Wayne Featuring 2 Chainz
no ID available for: Cashin' Out by Ca$h Out
no ID available for: I'm A Flirt by R. Kelly Or Bow Wow (Featuring T.I. & T-Pain)
no ID available for: For You I Will (Confidence) by Teddy Geiger
no ID available for: Down 4 U by Irv Gotti Presents The Inc. Featuring Ja Rule, Ashanti, Charli Baltimore & Vita
no ID available for: Oochie Wally by QB Finest Featuring Nas & Bravehearts
no ID available for: Come With Me by Puff Daddy Featuring Jimmy Page
no ID available for: Love You Down by INOJ/LATHUN
no ID available for: The Party Continues by JD Featuring Da Brat
no ID available for: Do Your Thing by 7 Mile
no ID available for: Ding-A-Ling by Hi-Town DJs
no ID available for: ESPN Presents The Jock Jam by Various Artists
no ID available 

In [17]:
song_id_dictionary

{0: {'title': 'Lose Control',
  'artist': 'Teddy Swims',
  'id': '17phhZDn6oGtzMe56NuWvj'},
 1: {'title': 'Stick Season',
  'artist': 'Noah Kahan',
  'id': '0mflMxspEfB0VbI1kyLiAv'},
 2: {'title': 'Beautiful Things',
  'artist': 'Benson Boone',
  'id': '3xkHsmpQCBMytMJNiDf3Ii'},
 3: {'title': 'Houdini', 'artist': 'Dua Lipa', 'id': '6D8y7Bck8h11byRY88Pt2z'},
 4: {'title': 'I Remember Everything',
  'artist': 'Zach Bryan Featuring Kacey Musgraves',
  'id': '4KULAymBBJcPRpk1yO4dOG'},
 5: {'title': 'Saturn', 'artist': 'SZA', 'id': '1bjeWoagtHmUKputLVyDxQ'},
 6: {'title': 'Feather',
  'artist': 'Sabrina Carpenter',
  'id': '2Zo1PcszsT9WQ0ANntJbID'},
 7: {'title': 'Austin', 'artist': 'Dasha', 'id': '2uqYupMHANxnwgeiXTZXzd'},
 8: {'title': 'Lovin On Me',
  'artist': 'Jack Harlow',
  'id': '4xhsWYTOGcal8zt0J161CU'},
 9: {'title': 'Too Sweet', 'artist': 'Hozier', 'id': '76JKIsdKrAfWUMjaA0u7v5'},
 10: {'title': 'Like That',
  'artist': 'Future, Metro Boomin & Kendrick Lamar',
  'id': '2tudvzsrR5

In [27]:
# converting the id dictionary into a dataframe and merging to the main csv dataframe already existing
id_df = pd.DataFrame(song_id_dictionary)
id_df = id_df.transpose()

final_df = pd.DataFrame()
final_df = pd.merge(csv_df, id_df, how = 'outer', left_index = True, right_index = True)
final_df = final_df.drop(columns = ['title_y', 'artist_y', 'Unnamed: 0.1'])
final_df = final_df.rename(columns = {"title_x": "title", "artist_x": "artist", "Unnamed: 0": "rank"})

final_df.head(4500)

,rank,title,artist,count,year,id
0,0,Lose Control,Teddy Swims,43,2024,17phhZDn6oGtzMe56NuWvj
1,1,Stick Season,Noah Kahan,40,2024,0mflMxspEfB0VbI1kyLiAv
2,2,Beautiful Things,Benson Boone,39,2024,3xkHsmpQCBMytMJNiDf3Ii
3,3,Houdini,Dua Lipa,37,2024,6D8y7Bck8h11byRY88Pt2z
4,4,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,35,2024,4KULAymBBJcPRpk1yO4dOG
...,...,...,...,...,...,...
4495,95,How Do I Survive,Amy Holland,16,1980,18ELywiVACeSIhIa2EqJ47
4496,96,Make A Little Magic,The Dirt Band,16,1980,728MAA4aiTVtTxn0Z2k1cu
4497,97,Old-Fashion Love,Commodores,16,1980,4uCZ87lJrOyc1L38MpaA08
4498,98,The Wanderer,Donna Summer,15,1980,2tD4VPcIx0QPXphgKc73Gz


In [28]:
# exporting dataframe into a new csv
final_df.to_csv('all_top100songs_by_year_with_ids.csv', index = False)